In [3]:
import numpy as np

dec_data = np.loadtxt('C:\\Users\\SAI CHARAN\\OneDrive\\Desktop\\proj\\Train_Dst_NoAuction_DecPre_CF_7.txt')

dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('C:\\Users\\SAI CHARAN\\OneDrive\\Desktop\\proj\\Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('C:\\Users\\SAI CHARAN\\OneDrive\\Desktop\\proj\\Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('C:\\Users\\SAI CHARAN\\OneDrive\\Desktop\\proj\\Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_data.shape)
print(dec_train.shape)
print(dec_val.shape)
print(dec_test.shape)


(149, 254750)
(149, 203800)
(149, 50950)
(149, 139587)


In [4]:
# load packages
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import to_categorical



In [5]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, LSTM, Reshape, Dense, Dropout
from keras.layers import LeakyReLU, concatenate
from keras.optimizers import Adam


In [6]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)
    dY = np.array(Y)
    dataY = dY[T - 1:N]
    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]
    return dataX.reshape(dataX.shape + (1,)), dataY

def prepare_x_y(data, k, T):
    x = prepare_x(data)
    y = get_label(data)
    x, y = data_classification(x, y, T=T)
    y = y[:,k] - 1
    y = to_categorical(y, 3)
    return x, y

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], input_shape[-1]),
                                 initializer='random_normal',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # Calculate attention scores
        e = tf.keras.backend.dot(x, self.W)

        # Calculate attention weights using softmax activation
        a = tf.keras.activations.softmax(e, axis=-2)

        # Apply attention weights to the input sequence
        output = x * a

        # Sum the weighted inputs to get the attention output
        output = tf.keras.backend.sum(output, axis=-2)

        return output

    def compute_output_shape(self, input_shape):
        return input_shape

In [22]:
k = 4 # which prediction horizon
T = 10 # the length of a single input i. e lookback_timestep THIS HYPERPARAMETER IS FINE TUNED FOR MODEL CONVERGENCE
n_hiddens = 64
# checkpoint_filepath = 'C:\\Users\\SAI CHARAN\\OneDrive\\Desktop\\proj\\path_to_your_BackADLOBmodel_checkpoints'
checkpoint_filepath = 'C:\\Users\\SAI CHARAN\\OneDrive\\Desktop\\proj\\path_to_your_BackADLOBmodel_checkpoints\\model.weights.h5'


In [9]:
print(dec_train.shape)
trainX_CNN, trainY_CNN = prepare_x_y(dec_train, k, T)
print(trainX_CNN.shape, trainY_CNN.shape)

(149, 203800)
(203791, 10, 40, 1) (203791, 3)


In [10]:
print(dec_test.shape)
testX_CNN, testY_CNN = prepare_x_y(dec_test, k, T)
print(testX_CNN.shape, testY_CNN.shape)

(149, 139587)
(139578, 10, 40, 1) (139578, 3)


In [11]:
print(dec_val.shape)
valX_CNN, valY_CNN = prepare_x_y(dec_val, k, T)
print(valX_CNN.shape, valY_CNN.shape)

(149, 50950)
(50941, 10, 40, 1) (50941, 3)


In [12]:
print(trainX_CNN.shape, trainY_CNN.shape)
print(valX_CNN.shape, valY_CNN.shape)
print(testX_CNN.shape, testY_CNN.shape)

(203791, 10, 40, 1) (203791, 3)
(50941, 10, 40, 1) (50941, 3)
(139578, 10, 40, 1) (139578, 3)


In [49]:
np.save('testX_CNN.npy', testX_CNN)

In [13]:
for i in range(10):
  print((testX_CNN[i][0][0]+testX_CNN[i][0][0])/2)


[0.2666]
[0.2669]
[0.2665]
[0.2669]
[0.2665]
[0.2669]
[0.2669]
[0.267]
[0.267]
[0.267]


In [14]:
def create_deeplob(T, NF, number_of_lstm):
    input_lmd = Input(shape=(T, NF, 1))

    # build the convolutional block
    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(input_lmd)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = Conv2D(32, (1, 10))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = Conv2D(32, (4, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    # build the inception module
    convsecond_1 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
    convsecond_1 = Conv2D(64, (3, 1), padding='same')(convsecond_1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)

    convsecond_2 = Conv2D(64, (1, 1), padding='same')(conv_first1)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
    convsecond_2 = Conv2D(64, (5, 1), padding='same')(convsecond_2)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)

    convsecond_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
    convsecond_3 = Conv2D(64, (1, 1), padding='same')(convsecond_3)
    convsecond_3 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_3)

    convsecond_output = keras.layers.concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3)
    conv_reshape = Reshape((int(convsecond_output.shape[1]), int(convsecond_output.shape[3])))(convsecond_output)
    conv_reshape = keras.layers.Dropout(0.2, noise_shape=(None, 1, int(conv_reshape.shape[2])))(conv_reshape, training=True)

    # build the last LSTM layer
    conv_lstm = LSTM(number_of_lstm, return_sequences=True)(conv_reshape)

    # add an attention layer
    attention_output = AttentionLayer()(conv_lstm)

    # build the output layer
    out = Dense(3, activation='softmax')(attention_output)

    model = Model(inputs=input_lmd, outputs=out)
    # adam = keras.optimizers.Adam(lr=0.0001)
    adam = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

deeplob = create_deeplob(trainX_CNN.shape[1], trainX_CNN.shape[2], n_hiddens)
deeplob.summary()

C:\Users\SAI CHARAN\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10, 40, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 10, 20,    │         96 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 10, 20,    │          0 │ conv2d[0][0]      │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 10, 20,    │      4,128 │ leaky_re_lu[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 10, 20,    │          0 │ conv2d_1[0][0]    │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 10, 20,    │      4,128 │ leaky_re_lu_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 10, 20,    │          0 │ conv2d_2[0][0]    │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 10, 10,    │      2,080 │ leaky_re_lu_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 10, 10,    │          0 │ conv2d_3[0][0]    │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 10, 10,    │      4,128 │ leaky_re_lu_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 10, 10,    │          0 │ conv2d_4[0][0]    │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 10, 10,    │      4,128 │ leaky_re_lu_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_5       │ (None, 10, 10,    │          0 │ conv2d_5[0][0]    │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 10, 1, 32) │     10,272 │ leaky_re_lu_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_6       │ (None, 10, 1, 32) │          0 │ conv2d_6[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 10, 1, 32) │      4,128 │ leaky_re_lu_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_7       │ (None, 10, 1, 32) │          0 │ conv2d_7[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 146,531 (572.39 KB)

 Trainable params: 146,531 (572.39 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
%%time

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

deeplob.fit(trainX_CNN, trainY_CNN, validation_data=(valX_CNN, valY_CNN),
            epochs=50, batch_size=128, verbose=2, callbacks=[model_checkpoint_callback])


Epoch 1/50
1593/1593 - 68s - 43ms/step - accuracy: 0.4194 - loss: 1.0230 - val_accuracy: 0.3726 - val_loss: 1.0886
Epoch 2/50
1593/1593 - 63s - 39ms/step - accuracy: 0.4478 - loss: 0.9560 - val_accuracy: 0.3726 - val_loss: 1.0909
Epoch 3/50
1593/1593 - 61s - 38ms/step - accuracy: 0.4677 - loss: 0.9158 - val_accuracy: 0.3749 - val_loss: 1.0744
Epoch 4/50
1593/1593 - 61s - 38ms/step - accuracy: 0.4774 - loss: 0.9021 - val_accuracy: 0.4635 - val_loss: 1.0102
Epoch 5/50
1593/1593 - 54s - 34ms/step - accuracy: 0.4951 - loss: 0.8787 - val_accuracy: 0.4789 - val_loss: 0.9645
Epoch 6/50
1593/1593 - 61s - 39ms/step - accuracy: 0.5083 - loss: 0.8665 - val_accuracy: 0.4772 - val_loss: 0.9639
Epoch 7/50
1593/1593 - 54s - 34ms/step - accuracy: 0.5524 - loss: 0.8416 - val_accuracy: 0.5082 - val_loss: 0.9157
Epoch 8/50
1593/1593 - 53s - 33ms/step - accuracy: 0.6217 - loss: 0.7848 - val_accuracy: 0.5051 - val_loss: 1.0068
Epoch 9/50
1593/1593 - 53s - 33ms/step - accuracy: 0.6693 - loss: 0.7304 - val_a

In [24]:
deeplob.load_weights(checkpoint_filepath)
pred = deeplob.predict(testX_CNN)

4362/4362 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step


In [48]:
import numpy as np

# Assuming your array is named `my_array`
# my_array = np.array([1, 2, 3, 4, 5])
pred.shape
p = [[]]
for i in range(100) :
    predicted_label = np.zeros_like(pred[i])
    predicted_label[np.argmax(pred[i])] = 1
    p.append(predicted_label)
print(p)
# Save the array to a file
# np.save('pred.npy', pred)


[[], array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([0., 0., 1.], dtype=float32), array([1., 0., 0.], dtype=float32), array([1., 0., 0.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dtype=float32), array([0., 0., 1.], dty

In [27]:
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

In [28]:
print('accuracy_score:', accuracy_score(np.argmax(testY_CNN, axis=1), np.argmax(pred, axis=1)))
print(classification_report(np.argmax(testY_CNN, axis=1), np.argmax(pred, axis=1), digits=4))

accuracy_score: 0.772141741535199
              precision    recall  f1-score   support

           0     0.7534    0.7845    0.7687     47976
           1     0.8423    0.7543    0.7959     48051
           2     0.7274    0.7782    0.7519     43551

    accuracy                         0.7721    139578
   macro avg     0.7744    0.7723    0.7722    139578
weighted avg     0.7759    0.7721    0.7728    139578



In [65]:
t = len(testY_CNN)
# D,N,U
money= 800
num_shares = 0
mid_price = 0
val = 0
for i in range(0,len(testX_CNN),200):
    # actual_label = testY_CNN[i]
    predicted_label = np.zeros_like(pred[i])
    predicted_label[np.argmax(pred[i])] = 1
    mid_price = (testX_CNN[i][0][0]+testX_CNN[i][0][0])/2
    if predicted_label[2]==1:
      if(num_shares) :
       money = num_shares*mid_price
       num_shares = 0
    else :
      if(money):
       num_shares = money/mid_price
       money = 0
    val = money
    if money ==0 :
      val = num_shares*mid_price
    # time.sleep(0.001)
print(val)

[2602.10046804]


In [36]:
from flask import Flask, render_template, request, Response,redirect, url_for,Response
import time
import json

app = Flask(__name__)

# Sample list of numbers
numbers = [10, 20, 19, 70, 60, 40, 30, 12, 11, 10, 9, 8]
index = 0
money = 0  # Initialize money

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/submit_number', methods=['POST'])
def submit_number():
    global money
    money = int(request.form['number'])
    return redirect(url_for('result'))

@app.route('/result')
def result():
    return render_template('result.html')

@app.route('/data')
def data():
    def generate_numbers():
        global index
        global money  # Access the global money variable
        while index < len(numbers):
            time.sleep(1)  # Simulate delay of 1 second
            yield 'data: {}\n\n'.format(json.dumps({'number': numbers[index]}))
            index += 1

    return Response(generate_numbers(), mimetype='text/event-stream')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on



 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [40]:
from flask import Flask, render_template, request, Response, redirect, url_for
import time
import json
from pyngrok import ngrok

app = Flask(__name__)

# Sample list of numbers
numbers = [10, 20, 19, 70, 60, 40, 30, 12, 11, 10, 9, 8]
index = 0
money = 0  # Initialize money

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/submit_number', methods=['POST'])
def submit_number():
    global money
    money = int(request.form['number'])
    return redirect(url_for('result'))

@app.route('/result')
def result():
    return render_template('result.html')

@app.route('/data')
def data():
    def generate_numbers():
        global index
        global money  # Access the global money variable
        while index < len(numbers):
            time.sleep(1)  # Simulate delay of 1 second
            yield 'data: {}\n\n'.format(json.dumps({'number': numbers[index]}))
            index += 1

    return Response(generate_numbers(), mimetype='text/event-stream')

# Start ngrok tunnel
public_url = ngrok.connect(port=5000)
print(" * ngrok tunnel:", public_url)

if __name__ == '__main__':
    app.run(debug=True)


t=2024-04-30T15:55:02+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2024-04-30T15:55:02+0530 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2024-04-30T15:55:02+0530 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [39]:
my_variable = 100